<a href="https://colab.research.google.com/github/Jack3690/Rad/blob/master/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def relu(z): 
  return np.max(0,z),z
 
def sigmoid(z): 
  return 1/(1+np.exp(-z)),z

def relu_backward(dA,z):
  gz=np.maximum(0,Z/Z)
  return dA*gz,z

def sigmoid_backward(dA,z):
  s=sigmoid(z)
  gz=s*(1-s)
  return dA*gz,z


In [ ]:
def L_initialize(layer_dims):

  parameters={}
  np.random.seed(3)
  L=len(layer_dims)
  for l in range(1,L):
 
    parameters['W'+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*np.sqrt(2/layer_dims[l-1])
    parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
  
  return parameters

In [ ]:
def linear_forward(A_prev,W,b):
  z=W@A_prev+b
  cache=(A_prev,W,b)

  return z,cache

def linear_activation_forward(A_prev, W, b, activation):

  if activation=="relu":
    z,linear_cache = linear_forward(A_prev,W,b)
    A,activation_cache=relu(z)

  elif activation=="sigmoid":
    z,linear_cache = linear_forward(A_prev,W,b)
    A,activation_cache=sigmoid(z)
  
  cache=(linear_cache,activation_cache)

  return A,cache

def L_model_forward(X,parameters):

  caches=[]
  A=X
  L=len(parameters)//2

  for l in range(1,L):
    A_prev=A
    A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
    caches.append(cache)

  AL, cache = linear_activation_forward(A, parameters["W"+str(L)],parameters["b"+str(L)], "sigmoid")
  caches.append(cache)

  return AL,caches

In [ ]:
def compute_cost(AL, Y):

    m = Y.shape[1]
    cost = -np.sum(np.multiply(np.log(AL),Y)+np.multiply(np.log(1-AL),1-Y))/m
    cost = np.squeeze(cost)      
    return cost

In [ ]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)

    return dA_prev, dW, db

In [ ]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db =  linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [ ]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
 
    for l in reversed(range(L-1)):

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [ ]:
def update_parameters(parameters, grads, learning_rate):

    L = len(parameters) //2
    for l in range(L):
        parameters["W" + str(l+1)] -=learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] -=learning_rate*grads["db"+str(l+1)]

    return parameters

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    np.random.seed(1)
    costs = []
    parameters = initialize_parameters_deep(layers_dims)

    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X, parameters)
  
        cost = compute_cost(AL, Y)
  
        grads = L_model_backward(AL, Y, caches)

        parameters = update_parameters(parameters, grads, learning_rate)

        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
def predict(X,parameters,Y):
  AL,_=L_model_forward(X,parameters)
  prediction=np.mean(np.abs(AL-Y))
  return prediction
